In [54]:
import ecc.ecc as ecc
import ecc.util as util
from os import urandom, getcwd
import hashlib, hmac

In [86]:
msg = "This is a message" # This is the message we commit to
false_msg = "This is not the message we commited to"
print(msg.encode('utf-8'))

b'This is a message'


In [90]:
privkey = ecc.PrivateKey(int("f02055a7c7c7680cbb23dc76ca7fea3b851984b63d93d1d6ccf3d3a9d84a97e2", 16))
pubkey = privkey.point
print(pubkey.sec().hex())

03f679efe0d62cf1fc465edfab93e40e2341ec7be081e846271a519287bd9356e4


In [92]:
def format_msg(msg):
    # create the lnpbp1 msg
    prefix = hashlib.sha256(b"LNPBP1").digest()
    lnpbp1_msg = prefix + hashlib.sha256(b"tag").digest() + msg.encode('utf-8')
    return lnpbp1_msg

def create_commitment(pubkey, msg):
    # implementation of LNPBP 1 (single key only)
    lnpbp1_msg = format_msg(msg)
    # HMAC s and P to get the tweaking factor f
    hmac = hmac.digest(pubkey.sec(), lnpbp1_msg, hashlib.sha256)
    f = int.from_bytes(HMAC.digest(), 'big')
    # assert f < n
    try:
        assert f < ecc.N
    except:
        print("ERROR: tweak overflow secp256k1 order")
    # Compute a new PrivateKey with f as secret
    return ecc.PrivateKey(f)

def verify_commitment(pubkey, msg, commitment):
    candidate = create_commitment(pubkey, msg) 
    print(f"candidate commitment is {candidate.point.sec().hex()}")
    return candidate.point + pubkey == commitment

tweaking_factor = create_commitment(pubkey, msg)
print(f"tweaking_factor pubkey is {tweaking_factor.point.sec().hex()}")
tweaked_pubkey = tweaking_factor.point + h.point
print(f"tweaked_pubkey is {tweaked_pubkey.sec().hex()}")
try:
    assert verify_commitment(h.point, msg, tweaked_pubkey) == True
except:
    print("Verification failed")

tweaking_factor pubkey is 0261b4c9588d379d3c913ee51df0e6fb23aeab8414551856eb07e1f7bc8256ae69
tweaked_pubkey is 03d02adaf5b19f8d80cef22e1f6be4769bb9a2d711d4b3da5fe1044b764da1dd89
candidate commitment is 0261b4c9588d379d3c913ee51df0e6fb23aeab8414551856eb07e1f7bc8256ae69
